## GAAIMS: Predicting Multiple Sclerosis from Dynamics of Gait Variability Using an Instrumented Treadmill - A Machine Learning-Based Approach

### Person generalization ML models and analysis


### Package imports 

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import math
import os
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
import copy

import xgboost 
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.externals import joblib
from sklearn.utils import shuffle
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_recall_curve
from inspect import signature
from scipy import interp
from pyitlib import discrete_random_variable as drv
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GroupKFold
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, roc_auc_score, roc_curve
from sklearn.metrics import make_scorer
import itertools
from sklearn.model_selection import ParameterGrid, ParameterSampler
import warnings
import time
warnings.filterwarnings("ignore")

C:\Users\rk4\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [5]:
path = 'C:\\Users\\rk4\\Desktop\\GAIT\\'

In [6]:
#Reading the raw dataframe 
raw_df = pd.read_csv(path+'FinalCodes\\gait_features.csv', index_col = 0)
#Dropping the NaNs
raw_df.dropna(inplace = True)
#Resetting the index
raw_df.reset_index(inplace= True)
print ('Raw data shape: ', raw_df.shape)

#Reading the Size-N dataframe 
sizeN_df = pd.read_csv(path+'FinalCodes\\size_normalized_gait_features.csv', index_col = 0)
#Dropping the NaNs
sizeN_df.dropna(inplace = True)
#Resetting the index
sizeN_df.reset_index(inplace= True)
print ('Size-N data shape: ', sizeN_df.shape)

#Reading the Regress-N dataframe 
regressN_df = pd.read_csv(path+'FinalCodes\\mr_scaled_features_30controlsTrialW.csv', index_col = 0)
regressN_df.reset_index(inplace= True)
print('Regress-N data shape: ', regressN_df.shape)

#Delete the treadmill speeds as features since they are very very correlated with stride speed
#Also delete Butterfly plot y-direction features since COP_Y is not adjusted 
#Swing time and SS_L are the same
to_drop = ['tspeed_HSR', 'tspeed_MidSSR', 'tspeed_TOR', 'tspeed_HSL', 'tspeed_TOL', 'tspeed_MidSSL',  'Butterfly_y_abs', 
           'ButterflySQ_y', 'SS_L', 'index']
raw_df.drop(to_drop, axis = 1, inplace= True)
raw_df = shuffle(raw_df, random_state = 0)
print ('Raw data shape: ', raw_df.shape) #21 features + PID + Trial ID + Label = 24 features 

sizeN_df.drop(to_drop, axis = 1, inplace= True)
sizeN_df = shuffle(sizeN_df, random_state = 0)
print ('Size-N data shape: ', sizeN_df.shape) #21 features + PID + Trial ID + Label = 24 features 

regressN_df.drop(['index'], axis = 1, inplace = True)
regressN_df = shuffle(regressN_df, random_state = 0)
print('Regress-N data shape: ', regressN_df.shape)  #21 features + PID + Trial ID + Label = 24 features 

Raw data shape:  (3230, 34)
Size-N data shape:  (3230, 34)
Regress-N data shape:  (3230, 25)
Raw data shape:  (3230, 24)
Size-N data shape:  (3230, 24)
Regress-N data shape:  (3230, 24)


### Evaluation

In [ ]:
def evaluate(model, test_features, yoriginal_, ypredicted_):
    best_index = model.cv_results_['mean_test_accuracy'].argmax()
    print('best_params: ', model.cv_results_['params'][best_index])

    #Stride-wise metrics 
    stride_metrics_mean, stride_metrics_std = [], [] #Mean and SD of stride based metrics - Acc, P, R, F1, AUC (in order)
    scores={'accuracy': make_scorer(acc), 'precision':'precision', 'recall':'recall', 'f1': 'f1', 'auc': 'roc_auc'}
    for score in scores:
        stride_metrics_mean.append(model.cv_results_['mean_test_'+score][best_index])
        stride_metrics_std.append(model.cv_results_['std_test_'+score][best_index])
    print('Stride-based model performance (mean): ', stride_metrics_mean)
    print('Stride-based model performance (standard deviation): ', stride_metrics_std)
    n_folds = 5
    person_acc, person_p, person_r, person_f1, person_auc = [], [], [], [], []
    #For ROC curves 
    tpr_list = []
    base_fpr = np.linspace(0, 1, 101)

    for i in range(n_folds):
        #For each fold, there are 2 splits: test and train (in order) and we need to retrieve the index 
        #of only test set for required 5 folds (best index)
        temp = test_features.loc[yoriginal_[(best_index*10) + (2*i)].index] #True labels for the test strides in each fold
        temp['pred'] = ypredicted_[(best_index*10) + (2*i)] #Predicted labels for the strides in the test set in each fold

        #Correctly classified strides i.e. 1 if stride is correctly classified and 0 if otherwise
        temp['correct'] = (temp['Label']==temp['pred'])

        #Proportion of correctly classified strides
        proportion_strides_correct = temp.groupby('PID').aggregate({'correct': 'mean'})  

        proportion_strides_correct['True Label'] = temp[['PID', 'Label']].groupby('PID').first() 

        #Label for the person - 0=healthy, 1=MS patient
        proportion_strides_correct['Predicted Label'] = proportion_strides_correct['True Label']*\
        (proportion_strides_correct['correct']>0.5)+(1-proportion_strides_correct['True Label'])*\
        (proportion_strides_correct['correct']<0.5) 

        #Probability of class 1 - MS patient for AUC calculation
        proportion_strides_correct['prob_class1'] = (1-proportion_strides_correct['True Label'])*\
        (1-proportion_strides_correct['correct'])+ proportion_strides_correct['True Label']*proportion_strides_correct['correct'] 

        fpr, tpr, _ = roc_curve(proportion_strides_correct['True Label'], proportion_strides_correct['prob_class1'])
        tpr = interp(base_fpr, fpr, tpr)
        tpr[0] = 0.0
        tpr_list.append(tpr)

        #Person wise metrics for each fold 
        person_acc.append(accuracy_score(proportion_strides_correct['Predicted Label'], proportion_strides_correct['True Label']))
        person_p.append(precision_score(proportion_strides_correct['Predicted Label'], proportion_strides_correct['True Label']))
        person_r.append(recall_score(proportion_strides_correct['Predicted Label'], proportion_strides_correct['True Label']))
        person_f1.append(f1_score(proportion_strides_correct['Predicted Label'], proportion_strides_correct['True Label']))
        person_auc.append(roc_auc_score(proportion_strides_correct['True Label'], proportion_strides_correct['prob_class1']))

    #Mean and standard deviation for person-based metrics 
    person_means = [np.mean(person_acc), np.mean(person_p), np.mean(person_r), np.mean(person_f1), np.mean(person_auc)]
    person_stds = [np.std(person_acc), np.std(person_p), np.std(person_r), np.std(person_f1), np.std(person_auc)]
    print('Person-based model performance (mean): ', person_means)
    print('Person-based model performance (standard deviation): ', person_stds)

    return tpr_list, [stride_metrics_mean, stride_metrics_std, person_means, person_stds]

### ML models 

In [ ]:
def acc(y_true,y_pred):
    global yoriginal, ypredicted
    yoriginal.append(y_true)
    ypredicted.append(y_pred)
    accuracy = accuracy_score(y_true, y_pred)
    return accuracy

In [ ]:
#We do not use LDA/QDA since our features are not normally distributed 
def models(X, Y, model_name = 'random_forest'):
    '''
    X, Y, PID groups so that strides of each person are either in training or in testing set
    model: model_name
    '''
    Y_ = Y['Label'] #Dropping the PID
    groups_ = Y['PID']
    gkf = GroupKFold(n_splits=5) 
    scores={'accuracy': make_scorer(acc), 'precision':'precision', 'recall':'recall', 'f1': 'f1', 'auc': 'roc_auc'}
    
    if(model_name == 'random_forest'): #Random Forest
        grid = {
       'randomforestclassifier__n_estimators': [40,45,50],\
       'randomforestclassifier__max_depth' : [15,20,25,None],\
       'randomforestclassifier__class_weight': [None, 'balanced'],\
       'randomforestclassifier__max_features': ['auto','sqrt','log2', None],\
       'randomforestclassifier__min_samples_leaf':[1,2,0.1,0.05]
        }
        #For z-score scaling on training and use calculated coefficients on test set
        rf_grid = make_pipeline(StandardScaler(), RandomForestClassifier(random_state=0))
        grid_search = GridSearchCV(rf_grid, param_grid=grid, scoring=scores\
                           , n_jobs = 1, cv=gkf.split(X, Y_, groups=groups_), refit=False)
    
    if(model_name == 'adaboost'): #Adaboost
        ada_grid = make_pipeline(StandardScaler(), AdaBoostClassifier(random_state=0))
        grid = {
        'adaboostclassifier__n_estimators':[50, 75, 100, 125, 150],\
        'adaboostclassifier__learning_rate':[0.01,.1, 1, 1.5, 2]\
        }
        grid_search = GridSearchCV(ada_grid, param_grid=grid, scoring=scores\
                           , n_jobs = 1, cv=gkf.split(X, Y_, groups=groups_), refit=False)
        
    if(model_name == 'kernel_svm'): #RBF SVM
        svc_grid = make_pipeline(StandardScaler(), SVC(kernel = 'rbf', probability=True, random_state=0))
        grid = {
        'svc__gamma':[0.0001, 0.001, 0.1, 1, 10, ]\
        }
        grid_search = GridSearchCV(svc_grid, param_grid=grid, scoring=scores\
                           , n_jobs = 1, cv=gkf.split(X, Y_, groups=groups_), refit=False)

    if(model_name == 'gbm'): #GBM
        gbm_grid = make_pipeline(StandardScaler(), GradientBoostingClassifier(random_state=0))
        grid = {
        'gradientboostingclassifier__learning_rate':[0.15,0.1,0.05], \
        'gradientboostingclassifier__n_estimators':[50, 100, 150],\
        'gradientboostingclassifier__max_depth':[2,4,7],\
        'gradientboostingclassifier__min_samples_split':[2,4], \
        'gradientboostingclassifier__min_samples_leaf':[1,3],\
        'gradientboostingclassifier__max_features':['auto','sqrt','log2', None],\
        }
        grid_search = GridSearchCV(gbm_grid, param_grid=grid, scoring=scores\
                           , n_jobs = 1, cv=gkf.split(X, Y_, groups=groups_), refit=False)
    
    if(model_name=='xgboost'): #Xgboost
        xgb_grid = make_pipeline(StandardScaler(), xgboost.XGBClassifier(random_state=0))
        grid = {
            'xgbclassifier__min_child_weight': [1, 5],\
            'xgbclassifier__gamma': [0.1, 0.5, 1, 1.5, 2],\
            'xgbclassifier__subsample': [0.6, 0.8, 1.0],\
            'xgbclassifier__colsample_bytree': [0.6, 0.8, 1.0],\
            'xgbclassifier__max_depth': [5, 7, 8]
        }
        grid_search = GridSearchCV(xgb_grid, param_grid=grid, scoring=scores\
                           , n_jobs = 1, cv=gkf.split(X, Y_, groups=groups_), refit=False)
    
    if(model_name == 'knn'): #KNN
        knn_grid = make_pipeline(StandardScaler(), KNeighborsClassifier())
        grid = {
            'kneighborsclassifier__n_neighbors': [1, 3, 4, 5, 10],\
            'kneighborsclassifier__p': [1, 2, 3, 4, 5]\
        }
        grid_search = GridSearchCV(knn_grid, param_grid=grid, scoring=scores\
                           , n_jobs = 1, cv=gkf.split(X, Y_, groups=groups_), refit=False)
        
    if(model_name == 'decision_tree'): #Decision Tree
        dec_grid = make_pipeline(StandardScaler(), DecisionTreeClassifier(random_state=0))
        #For z-score scaling on training and use calculated coefficients on test set
        grid = {'decisiontreeclassifier__min_samples_split': range(2, 50)}
        grid_search = GridSearchCV(dec_grid, param_grid=grid, scoring=scores\
                           , n_jobs = 1, cv=gkf.split(X, Y_, groups=groups_), refit=False)

    if(model_name == 'linear_svm'): #Linear SVM
        lsvm_grid = make_pipeline(StandardScaler(), LinearSVC(random_state=0))
        grid = {
            'linearsvc__loss': ['hinge','squared_hinge'],\

        }
        grid_search = GridSearchCV(lsvm_grid, param_grid=grid, scoring=scores\
                           , n_jobs = 1, cv=gkf.split(X, Y_, groups=groups_), refit=False)
    
    if(model_name == 'logistic_regression'): #Logistic regression
        lr_grid = make_pipeline(StandardScaler(), LogisticRegression())
        grid = {
            'logisticregression__random_state': [0]}
            
        grid_search = GridSearchCV(lr_grid, param_grid=grid, scoring=scores\
                           , n_jobs = 1, cv=gkf.split(X, Y_, groups=groups_), refit=False)
    grid_search.fit(X, Y_, groups=groups_) #Fitting on the training set to find the optimal hyperparameters 
    tpr_list, stride_person_metrics = evaluate(grid_search, Y, yoriginal, ypredicted)
    return tpr_list, stride_person_metrics

### Raw data

In [ ]:
#CV for people generalize so no train-test split
X_raw = raw_df.drop(['Label', 'PID', 'TrialID'], axis = 1)
Y_raw = raw_df[['PID', 'Label']] #PID to compute person based metrics later 

#How to make sure test set has equal no. of MS and controls in each split??

In [ ]:
ml_models = ['random_forest', 'adaboost', 'kernel_svm', 'gbm', 'xgboost', 'knn', 'decision_tree',  'linear_svm', 
             'logistic_regression']
raw_metrics = pd.DataFrame(columns = ml_models) #Dataframe to store accuracies for each ML model for raw data 
#For storing predicted probabilities for person (for class 1) to show ROC curves 
tprs_raw = pd.DataFrame(columns = ml_models) 

for ml_model in ml_models:
    print (ml_model)
    yoriginal = []
    ypredicted = []
    tprs, stride_person_metrics = models(X_raw, Y_raw, ml_model)
    raw_metrics[ml_model] = sum(stride_person_metrics, [])
    tprs_raw[ml_model] = tprs
    print ('********************************')

raw_metrics.index = ['stride_mean_accuracy', 'stride_mean_precision', 'stride_mean_recall', 'stride_mean_F1', \
                     'stride_mean_AUC', 'stride_std_accuracy', 'stride_std_precision', 'stride_std_recall', 'stride_std_F1', \
                     'stride_std_AUC','person_mean_accuracy', 'person_mean_precision', 'person_mean_recall', 'person_mean_F1',\
                     'person_mean_AUC', 'person_std_accuracy', 'person_std_precision', 'person_std_recall', 'person_std_F1',\
                     'person_std_AUC']  
raw_metrics.to_csv(path+'..//person_generalize//person_generalize_results_raw_data.csv')
tprs_raw.to_csv(path+'..//person_generalize//person_generalize_ROCresults_raw_data.csv')

In [ ]:
raw_metrics

### Size-N data

In [ ]:
#CV for people generalize so no train-test split
X_sizeN = sizeN_df.drop(['Label', 'PID', 'TrialID'], axis = 1)
Y_sizeN = sizeN_df[['PID', 'Label']] #PID to compute person based metrics later 

In [ ]:
ml_models = ['random_forest', 'adaboost', 'kernel_svm', 'gbm', 'xgboost', 'knn', 'decision_tree',  'linear_svm', 
             'logistic_regression']
sizeN_metrics = pd.DataFrame(columns = ml_models) #Dataframe to store accuracies for each ML model for raw data 
#For storing predicted probabilities for person (for class 1) to show ROC curves 
tprs_sizeN = pd.DataFrame(columns = ml_models) 

for ml_model in ml_models:
    print (ml_model)
    yoriginal = []
    ypredicted = []
    tprs, stride_person_metrics = models(X_sizeN, Y_sizeN, ml_model)
    sizeN_metrics[ml_model] = sum(stride_person_metrics, [])
    tprs_sizeN[ml_model] = tprs
    print ('********************************')

sizeN_metrics.index = ['stride_mean_accuracy', 'stride_mean_precision', 'stride_mean_recall', 'stride_mean_F1', \
                     'stride_mean_AUC', 'stride_std_accuracy', 'stride_std_precision', 'stride_std_recall', 'stride_std_F1', \
                     'stride_std_AUC','person_mean_accuracy', 'person_mean_precision', 'person_mean_recall', 'person_mean_F1',\
                     'person_mean_AUC', 'person_std_accuracy', 'person_std_precision', 'person_std_recall', 'person_std_F1',\
                     'person_std_AUC']  
sizeN_metrics.to_csv(path+'..//person_generalize//person_generalize_results_sizeN_data.csv')
tprs_sizeN.to_csv(path+'..//person_generalize//person_generalize_ROCresults_sizeN_data.csv')

In [ ]:
sizeN_metrics.to_csv(path+'..//person_generalize//person_generalize_results_sizeN_data.csv')

In [ ]:
sizeN_metrics

### Investigate

In [4]:
def evaluate(model, test_features, yoriginal_, ypredicted_):
    prop_strides_appended = pd.DataFrame()
    best_index = model.cv_results_['mean_test_accuracy'].argmax()
    print('best_params: ', model.cv_results_['params'][best_index])

    #Stride-wise metrics 
    stride_metrics_mean, stride_metrics_std = [], [] #Mean and SD of stride based metrics - Acc, P, R, F1, AUC (in order)
    scores={'accuracy': make_scorer(acc), 'precision':'precision', 'recall':'recall', 'f1': 'f1', 'auc': 'roc_auc'}
    for score in scores:
        stride_metrics_mean.append(model.cv_results_['mean_test_'+score][best_index])
        stride_metrics_std.append(model.cv_results_['std_test_'+score][best_index])
    print('Stride-based model performance (mean): ', stride_metrics_mean)
    print('Stride-based model performance (standard deviation): ', stride_metrics_std)
    n_folds = 7
    person_acc, person_p, person_r, person_f1, person_auc = [], [], [], [], []
    #For ROC curves 
    tpr_list = []
    base_fpr = np.linspace(0, 1, 101)

    for i in range(n_folds):
        #For each fold, there are 2 splits: test and train (in order) and we need to retrieve the index 
        #of only test set for required 5 folds (best index)
        temp = test_features.loc[yoriginal_[(best_index*2*n_folds) + (2*i)].index] #True labels for the test strides in each fold
        temp['pred'] = ypredicted_[(best_index*2*n_folds) + (2*i)] #Predicted labels for the strides in the test set in each fold

        #Correctly classified strides i.e. 1 if stride is correctly classified and 0 if otherwise
        temp['correct'] = (temp['Label']==temp['pred'])
        
        print (temp)
        #Proportion of correctly classified strides
        proportion_strides_correct = temp.groupby('PID').aggregate({'correct': 'mean'})  

        proportion_strides_correct['True Label'] = temp[['PID', 'Label']].groupby('PID').first() 
        
        print ('Proportion correct', proportion_strides_correct)
        prop_strides_appended= prop_strides_appended.append(proportion_strides_correct)
        print ('append', prop_strides_appended)
        #Label for the person - 0=healthy, 1=MS patient
        proportion_strides_correct['Predicted Label'] = proportion_strides_correct['True Label']*\
        (proportion_strides_correct['correct']>=0.495)+(1-proportion_strides_correct['True Label'])*\
        (proportion_strides_correct['correct']<0.495) 

        #Probability of class 1 - MS patient for AUC calculation
        proportion_strides_correct['prob_class1'] = (1-proportion_strides_correct['True Label'])*\
        (1-proportion_strides_correct['correct'])+ proportion_strides_correct['True Label']*proportion_strides_correct['correct'] 

        fpr, tpr, _ = roc_curve(proportion_strides_correct['True Label'], proportion_strides_correct['prob_class1'])
        tpr = interp(base_fpr, fpr, tpr)
        tpr[0] = 0.0
        tpr_list.append(tpr)

        #Person wise metrics for each fold 
        person_acc.append(accuracy_score(proportion_strides_correct['Predicted Label'], proportion_strides_correct['True Label']))
        person_p.append(precision_score(proportion_strides_correct['Predicted Label'], proportion_strides_correct['True Label']))
        person_r.append(recall_score(proportion_strides_correct['Predicted Label'], proportion_strides_correct['True Label']))
        person_f1.append(f1_score(proportion_strides_correct['Predicted Label'], proportion_strides_correct['True Label']))
        person_auc.append(roc_auc_score(proportion_strides_correct['True Label'], proportion_strides_correct['prob_class1']))

    #Mean and standard deviation for person-based metrics 
    person_means = [np.mean(person_acc), np.mean(person_p), np.mean(person_r), np.mean(person_f1), np.mean(person_auc)]
    person_stds = [np.std(person_acc), np.std(person_p), np.std(person_r), np.std(person_f1), np.std(person_auc)]
    print('Person-based model performance (mean): ', person_means)
    print('Person-based model performance (standard deviation): ', person_stds)
    prop_strides_appended.to_csv(path+'..//person_generalize//prop_strides_appeneded_adaboost.csv')
    return tpr_list, [stride_metrics_mean, stride_metrics_std, person_means, person_stds]

In [ ]:
#We do not use LDA/QDA since our features are not normally distributed        
#     mlp_grid = make_pipeline(StandardScaler(), MLPClassifier(hidden_layer_sizes=(50, 100, 100, 50, 30, 10, 5), activation='relu', solver='adam' , \
#                                                             learning_rate = 'adaptive', learning_rate_init=0.001, \
#                                                              shuffle=False, max_iter = 200))
#     #(50, 100, 100, 50, 30, 10, 5)
#     grid = {
#     'mlpclassifier__random_state': [0], 
#     }
#     grid_search = GridSearchCV(mlp_grid, param_grid=grid, scoring=scores\
#                    , n_jobs = 1, cv=gkf.split(X, Y_, groups=groups_), refit=False)  #0.79 good accuracy and F1 
    
#     ada_grid = make_pipeline(StandardScaler(), AdaBoostClassifier(random_state=0))
#     grid = {
#     'adaboostclassifier__base_estimator': [DecisionTreeClassifier(random_state=0, min_samples_split=17)], 
#     'adaboostclassifier__n_estimators':[120],#, 50, 75, 100, 125, 150],\
#     'adaboostclassifier__learning_rate':[0.8] #[0.1, 0.3, 0.5, 0.6, 0.7, ]
#     }
#     grid_search = GridSearchCV(ada_grid, param_grid=grid, scoring=scores\
#                    , n_jobs = 1, cv=gkf.split(X, Y_, groups=groups_), refit=False)  #0.79
      
#     ada_grid = make_pipeline(StandardScaler(), AdaBoostClassifier(random_state=0))
#     grid = {
#     'adaboostclassifier__base_estimator': [GradientBoostingClassifier(random_state=0, learning_rate = 0.8, n_estimators = 120)], 
#     'adaboostclassifier__n_estimators':[120],#, 50, 75, 100, 125, 150],\
#     'adaboostclassifier__learning_rate':[0.8] #[0.1, 0.3, 0.5, 0.6, 0.7, ]
#     }
#     grid_search = GridSearchCV(ada_grid, param_grid=grid, scoring=scores\
#                    , n_jobs = 1, cv=gkf.split(X, Y_, groups=groups_), refit=False) #0.81

#     ada_grid = make_pipeline(StandardScaler(), AdaBoostClassifier(random_state=0))
#     grid = {
#     'adaboostclassifier__base_estimator': [RandomForestClassifier(random_state=0)], 
#     'adaboostclassifier__n_estimators':[120],#, 50, 75, 100, 125, 150],\
#     'adaboostclassifier__learning_rate':[0.8] #[0.1, 0.3, 0.5, 0.6, 0.7, ]
#     }
#     grid_search = GridSearchCV(ada_grid, param_grid=grid, scoring=scores\
#                    , n_jobs = 1, cv=gkf.split(X, Y_, groups=groups_), refit=False) #0.79
    
    
#     gbm_grid = make_pipeline(StandardScaler(), GradientBoostingClassifier(random_state=0))
#     grid = {
        
#     'gradientboostingclassifier__learning_rate':[0.8], \
#     'gradientboostingclassifier__n_estimators':[120],\
#     'gradientboostingclassifier__max_depth':[3],\
#     'gradientboostingclassifier__min_samples_split':range(2, 5), \
# #     'gradientboostingclassifier__min_samples_leaf':[1,3],\
#     'gradientboostingclassifier__max_features':['auto','sqrt','log2', None],\
#     }
#     grid_search = GridSearchCV(gbm_grid, param_grid=grid, scoring=scores\
#                        , n_jobs = 1, cv=gkf.split(X, Y_, groups=groups_), refit=False) #0.76

#     grid = {
#     'randomforestclassifier__n_estimators': range(2, 40),\
# #     'randomforestclassifier__max_depth' : range(2, 10),\
#     'randomforestclassifier__max_features': ['auto','sqrt','log2', None],\
# #     'randomforestclassifier__min_samples_leaf':[1,2,0.1,0.05]
#     }
#     #For z-score scaling on training and use calculated coefficients on test set
#     rf_grid = make_pipeline(StandardScaler(), RandomForestClassifier(random_state=0))
#     grid_search = GridSearchCV(rf_grid, param_grid=grid, scoring=scores\
#                        , n_jobs = 1, cv=gkf.split(X, Y_, groups=groups_), refit=False) #0.74 
    
#     grid_search.fit(X, Y_, groups=groups_) #Fitting on the training set to find the optimal hyperparameters 
#     tpr_list, stride_person_metrics = evaluate(grid_search, Y, yoriginal, ypredicted)
#     return tpr_list, stride_person_metrics

### MLP investigate

In [7]:
def acc(y_true,y_pred):
    global yoriginal, ypredicted   
    yoriginal.append(y_true)
    ypredicted.append(y_pred)
#     print ('In acc: ', yoriginal, ypredicted)
    accuracy = accuracy_score(y_true, y_pred)
    return accuracy


def evaluate_mlp(test_features, yoriginal_, ypredicted_):
#     print (len(yoriginal_), len(ypredicted_))
    prop_strides_appended = pd.DataFrame()
    n_folds = 7
    person_acc= []
    for i in range(n_folds):
        #For each fold, there are 2 splits: test and train (in order) and we need to retrieve the index 
        #of only test set for required 5 folds (best index)
#         print ('index', yoriginal_[i].index)
        temp = test_features.loc[yoriginal_[i].index] #True labels for the test strides in each fold
        temp['pred'] = ypredicted_[i] #Predicted labels for the strides in the test set in each fold

        #Correctly classified strides i.e. 1 if stride is correctly classified and 0 if otherwise
        temp['correct'] = (temp['Label']==temp['pred'])
        
#         print (temp)
        #Proportion of correctly classified strides
        proportion_strides_correct = temp.groupby('PID').aggregate({'correct': 'mean'})  

        proportion_strides_correct['True Label'] = temp[['PID', 'Label']].groupby('PID').first() 
        
#         print ('Proportion correct', proportion_strides_correct)
        prop_strides_appended= prop_strides_appended.append(proportion_strides_correct)
#         print ('append', prop_strides_appended)
        #Label for the person - 0=healthy, 1=MS patient
        proportion_strides_correct['Predicted Label'] = proportion_strides_correct['True Label']*\
        (proportion_strides_correct['correct']>=0.495)+(1-proportion_strides_correct['True Label'])*\
        (proportion_strides_correct['correct']<0.495) 

        #Probability of class 1 - MS patient for AUC calculation
        proportion_strides_correct['prob_class1'] = (1-proportion_strides_correct['True Label'])*\
        (1-proportion_strides_correct['correct'])+ proportion_strides_correct['True Label']*proportion_strides_correct['correct'] 

        #Person wise metrics for each fold 
        person_acc.append(accuracy_score(proportion_strides_correct['Predicted Label'], proportion_strides_correct['True Label']))
    #Mean and standard deviation for person-based metrics 
    print ('Person results', np.mean(person_acc), np.std(person_acc))

In [8]:
X_regressN_investigate = regressN_df.drop(['Label', 'PID', 'TrialID'], axis = 1)
Y_regressN_investigate = regressN_df[['PID', 'Label']] #PID to compute person based metrics later 

In [ ]:
ml_models = ['MLP']
start = time.time()
print (ml_models)
X = X_regressN_investigate
Y = Y_regressN_investigate
Y_ = Y['Label'] #Dropping the PID
groups_ = Y['PID']
gkf = GroupKFold(n_splits=7) 
#     scores={'accuracy': make_scorer(acc), 'precision':'precision', 'recall':'recall', 'f1': 'f1', 'auc': 'roc_auc'}

grid = {
'random_state': [0], 
'hidden_layer_sizes': [x for x in itertools.product((21, 42, 84, 7, 5, 120),repeat=7)]
}

scorer = make_scorer(acc)

sampler = ParameterGrid(grid)
for params in sampler:
    yoriginal=[]
    ypredicted=[]
    mlp_grid = make_pipeline(StandardScaler(), MLPClassifier(activation='relu', solver='adam',\
                                                       learning_rate = 'adaptive', learning_rate_init=0.001, 
                                                        shuffle=False, max_iter = 200, **params))
    scores = []
    for ix_train, ix_test in gkf.split(X, Y_, groups=groups_): 
        clf_fitted = mlp_grid.fit(X.iloc[ix_train], Y_.iloc[ix_train])
        score = scorer(clf_fitted, X.iloc[ix_test], Y_.iloc[ix_test])

    #             print ('At this fold: ', score)
        scores.append(score)
    print ('params = ', params)
    print ('Stride accuracy = ', np.mean(scores), np.std(scores))
#     print (len(yoriginal), len(ypredicted))
    evaluate_mlp(Y, yoriginal, ypredicted)

print ('********************************')
print (time.time()-start)

In [ ]:
#The max I have is 0.71 right now. So any person accuracy > 0.71 is better 

### Regress-N data

In [ ]:
#CV for people generalize so no train-test split
X_regressN = regressN_df.drop(['Label', 'PID', 'TrialID'], axis = 1)
Y_regressN = regressN_df[['PID', 'Label']] #PID to compute person based metrics later 

In [ ]:
ml_models = ['random_forest', 'adaboost', 'kernel_svm', 'gbm', 'xgboost', 'knn', 'decision_tree',  'linear_svm', 
             'logistic_regression']
regressN_metrics = pd.DataFrame(columns = ml_models) #Dataframe to store accuracies for each ML model for raw data 
#For storing predicted probabilities for person (for class 1) to show ROC curves 
tprs_regressN = pd.DataFrame(columns = ml_models) 

for ml_model in ml_models:
    print (ml_model)
    yoriginal = []
    ypredicted = []
    tprs, stride_person_metrics = models(X_regressN, Y_regressN, ml_model)
    regressN_metrics[ml_model] = sum(stride_person_metrics, [])
    tprs_regressN[ml_model] = tprs
    print ('********************************')

regressN_metrics.index = ['stride_mean_accuracy', 'stride_mean_precision', 'stride_mean_recall', 'stride_mean_F1', \
                     'stride_mean_AUC', 'stride_std_accuracy', 'stride_std_precision', 'stride_std_recall', 'stride_std_F1', \
                     'stride_std_AUC','person_mean_accuracy', 'person_mean_precision', 'person_mean_recall', 'person_mean_F1',\
                     'person_mean_AUC', 'person_std_accuracy', 'person_std_precision', 'person_std_recall', 'person_std_F1',\
                     'person_std_AUC']  
regressN_metrics.to_csv(path+'..//person_generalize//person_generalize_results_regressN_data.csv')
tprs_regressN.to_csv(path+'..//person_generalize//person_generalize_ROCresults_regressN_data.csv')

In [ ]:
regressN_metrics

### ROC curves

In [ ]:
base_fpr = np.linspace(0, 1, 101)
ml_models = ['random_forest', 'adaboost', 'kernel_svm', 'gbm', 'xgboost', 'decision_tree',  'linear_svm', 
             'logistic_regression'] #'knn'
ml_model_names = {'random_forest': 'RF', 'adaboost': 'Adaboost', 'kernel_svm': 'RBF SVM', 'gbm': 'GBM', \
                  'xgboost': 'Xgboost', 'knn': 'KNN', 'decision_tree': 'DT',  'linear_svm': 'LSVM', 
             'logistic_regression': 'LR'}

fig, axes = plt.subplots(3, 1, sharex=True, sharey = True, figsize=(5, 9))
sns.despine(offset=0)

# #Raw Data 
# axes[0].plot([0, 1], [0, 1], linestyle='--', label='Majority (AUC = 0.5)', linewidth = 2, color = 'k')
# for ml_model in ml_models:
#     tprs = tprs_raw[ml_model] # person-based prediction probabilities
#     tprs = np.array(tprs)
#     mean_tprs = tprs.mean(axis=0)
#     std = tprs.std(axis=0)

#     tprs_upper = np.minimum(mean_tprs + std, 1)
#     tprs_lower = mean_tprs - std
#     axes[0].fill_between(base_fpr, tprs_lower, tprs_upper, color='grey', alpha=0.3)
#     axes[0].plot(base_fpr, mean_tprs, label=ml_model_names[ml_model]+' (AUC = '+ str(round(raw_metrics.loc['person_mean_AUC']
#                      [ml_model], 2)) + r'$\pm$' + str(round(raw_metrics.loc['person_std_AUC']
#                      [ml_model], 2)) + ')', linewidth = 2)
# axes[0].set_ylabel('True Positive Rate')
# axes[0].legend(loc='upper center', bbox_to_anchor=(1.27, 1), ncol=1)
# axes[0].set_title('Raw data')

#SizeN Data 
axes[1].plot([0, 1], [0, 1], linestyle='--', label='Majority (AUC = 0.5)', linewidth = 2, color = 'k')
for ml_model in ml_models:
    tprs = tprs_sizeN[ml_model] # person-based prediction probabilities
    tprs = np.array(tprs)
    mean_tprs = tprs.mean(axis=0)
    std = tprs.std(axis=0)

    tprs_upper = np.minimum(mean_tprs + std, 1)
    tprs_lower = mean_tprs - std
#     axes[1].fill_between(base_fpr, tprs_lower, tprs_upper, color='grey', alpha=0.3)
    axes[1].plot(base_fpr, mean_tprs, label=ml_model_names[ml_model]+' (AUC = '+ str(round(sizeN_metrics.loc['person_mean_AUC']
                     [ml_model], 2)) + r'$\pm$' + str(round(sizeN_metrics.loc['person_std_AUC']
                     [ml_model], 2)) + ')', linewidth = 2)
axes[1].legend(loc='upper center', bbox_to_anchor=(1.27, 1), ncol=1)
axes[1].set_ylabel('True Positive Rate')
axes[1].set_title('SizeN data')

#RegressN Data 
axes[2].plot([0, 1], [0, 1], linestyle='--', label='Majority (AUC = 0.5)', linewidth = 2, color = 'k')
for ml_model in ml_models:
    tprs = tprs_regressN[ml_model] # person-based prediction probabilities
    tprs = np.array(tprs)
    mean_tprs = tprs.mean(axis=0)
    std = tprs.std(axis=0)

    tprs_upper = np.minimum(mean_tprs + std, 1)
    tprs_lower = mean_tprs - std
#     axes[2].fill_between(base_fpr, tprs_lower, tprs_upper, color='grey', alpha=0.3)
    axes[2].plot(base_fpr, mean_tprs, label=ml_model_names[ml_model]+' (AUC = '+ str(round(regressN_metrics.loc['person_mean_AUC']
                     [ml_model], 2)) + r'$\pm$' + str(round(regressN_metrics.loc['person_std_AUC']
                     [ml_model], 2)) + ')', linewidth = 2)
axes[2].set_ylabel('True Positive Rate')
axes[2].set_title('RegressN data')
axes[2].legend(loc='upper center', bbox_to_anchor=(1.27, 1), ncol=1)

axes[2].set_xlabel('False Positive Rate')
plt.savefig(path + '..//person_generalize//ROC_person_generalize.png', dpi = 250)
plt.show()